In [11]:
import tensorflow as tf
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh
import scipy.sparse as sp
import pandas as pd
import pickle
from datetime import timedelta
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr
#from tensorflow.contrib.rnn.python.ops import rnn_cell as RNNCell
from tensorflow.python.ops.rnn_cell_impl import _RNNCell as RNNCell
import collections
from tensorflow.contrib import rnn
import h5py
#from tensorflow.python.ops.rnn_cell_impl import _RNNCell 
import random

In [12]:
def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    #adj[np.isnan(adj)] = 0.
    adj = tf.abs(adj)
    rowsum = tf.reduce_sum(adj, 1)# sum by row

    d_inv_sqrt = tf.pow(rowsum, -0.5)
   
    #d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    
    d_mat_inv_sqrt = tf.diag(d_inv_sqrt)

    return tf.matmul(tf.matmul(d_mat_inv_sqrt, adj), d_mat_inv_sqrt)

def masked_mae_tf(preds, labels, null_val=np.nan):
    """
    Accuracy with masking.
    :param preds:
    :param labels:
    :param null_val:
    :return:
    """
    if np.isnan(null_val):
        mask = ~tf.is_nan(labels)
    else:
        mask = tf.not_equal(labels, null_val)
    mask = tf.cast(mask, tf.float32)
    mask /= tf.reduce_mean(mask)
    mask = tf.where(tf.is_nan(mask), tf.zeros_like(mask), mask)
    loss = tf.abs(tf.subtract(preds, labels))
    loss = loss * mask
    loss = tf.where(tf.is_nan(loss), tf.zeros_like(loss), loss)
    return tf.reduce_mean(loss)

In [13]:
class StandardScaler:
    """
    Standard the input
    """

    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def transform(self, data):
        return (data - self.mean) / self.std

    def inverse_transform(self, data):
        return (data * self.std) + self.mean


In [14]:
# install tables
raw_data = pd.read_hdf('../../data/METR-LA/metr-la.h5')

In [15]:
raw_data.shape

(34272, 207)

In [16]:
raw_data.head()

,773869,767541,767542,717447,717446,717445,773062,767620,737529,717816,...,772167,769372,774204,769806,717590,717592,717595,772168,718141,769373
2012-03-01 00:00:00,64.375000,67.625000,67.125000,61.500000,66.875000,68.750000,65.125,67.125,59.625000,62.750000,...,45.625000,65.500,64.500000,66.428571,66.875,59.375000,69.000000,59.250000,69.000000,61.875
2012-03-01 00:05:00,62.666667,68.555556,65.444444,62.444444,64.444444,68.111111,65.000,65.000,57.444444,63.333333,...,50.666667,69.875,66.666667,58.555556,62.000,61.111111,64.444444,55.888889,68.444444,62.875
2012-03-01 00:10:00,64.000000,63.750000,60.000000,59.000000,66.500000,66.250000,64.500,64.250,63.875000,65.375000,...,44.125000,69.000,56.500000,59.250000,68.125,62.500000,65.625000,61.375000,69.857143,62.000
2012-03-01 00:15:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000
2012-03-01 00:20:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000


In [17]:
# Create model
def gcn(x, weights, biases, batch_size, n_input, frequency,flag, n_output_vec):
    # Hidden layer with RELU activation
    
   # output_list = tf.Variable(tf.zeros([n_output_vec,1]),dtype=tf.float32) #'Tensor' object does not support item assignment, cant build Ypre

    #Xtem = tf.reshape(x[i,:], [n_input, frequency])
    #Xtem = tf.transpose(Xtem)
    #Atem = tf.convert_to_tensor(A_whole_final, dtype=np.float32)
    #Atem1 = tf.diag(tf.ones([n_input])) # Atem not palying any roles
    #Ytem = tf.reshape(Y[i,:], [n_input, 1])
    #Atem = tf.diag(tf.ones([n_input]))
    #x = tf.reshape(x, [-1, sn, 1]) # 100, 207, 1
    
    # x (?, 207, 12)
    x = tf.transpose(x, [1, 0, 2]) # 207, ?, 12
    x = tf.reshape(x, [sn, -1]) # 207, batch*feature_num
    Atem1 = 0.5*(weights['A1'] + tf.transpose(weights['A1']))#+ Atem 
    Atem1 = normalize_adj(Atem1)
    #th = tf.constant(0.01, dtype=tf.float32)
    #where = tf.subtract(Atem1, th)
    #Atem1 = tf.nn.relu(where)

    Z1 = tf.matmul(Atem1, x) # 207, batch*feature_num  #+ tf.matmul( tf.matmul(weights['A1'], weights['A1']), Xtem)
    Z1 = tf.reshape(Z1, [-1, frequency]) # 207* 100, frequency
    #layer_1 = tf.matmul(Z1, weights['h1']) 
    layer_1 = tf.add(tf.matmul(Z1, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1) # 207*100, hidden1

    
    Atem2 = 0.5*(weights['A2'] + tf.transpose(weights['A2']))#+ Atem 
    Atem2 = normalize_adj(Atem2)
    
    layer_1 = tf.reshape(layer_1, [sn, -1])  # 207, batchsize*hidden1
    Z2 = tf.matmul(Atem2, layer_1)
    Z2 = tf.reshape(Z2, [-1, n_hidden_vec1]) # 207*batchsize, n_hidden_vec1
    layer_2 = tf.add(tf.matmul(Z2, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2) # 207*batchsize, hidden2

    Atem3 = 0.5*(weights['A3'] + tf.transpose(weights['A3']))#+ Atem 
    Atem3 = normalize_adj(Atem3) 
    
    layer_2 = tf.reshape(layer_2, [sn, -1])  # 207, batchsize*hidden2
    Z3 = tf.matmul(Atem3, layer_2)
    Z3 = tf.reshape(Z3, [-1, n_hidden_vec2]) # 207*batchsize, hidden2
    layer_3 = tf.add(tf.matmul(Z3, weights['h3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3) # 207*batchsize, hidden3
    
    #Atem4 = 0.5*(weights['A4'] + tf.transpose(weights['A4']))#+ Atem 
    #Atem4 = normalize_adj(Atem4)
    #Z4 = tf.matmul(Atem4, layer_3)
    #layer_4 = tf.add(tf.matmul(Z4, weights['h4']), biases['b4'])
    #layer_4 = tf.nn.relu(layer_4)

    # flattern
    #layer_3 = tf.reshape(layer_3, [1, 272*n_hidden_vec3])

    #F1 = tf.add(tf.matmul(layer_3, weights['f1']), biases['bf1'])
    #F1 = tf.nn.relu(F1)

    #F2 = tf.add(tf.matmul(F1, weights['f2']), biases['bf2'])
    #F2 = tf.nn.relu(F2)

    #F3 = tf.add(tf.matmul(F1, weights['f3']), biases['bf3'])
    #out_layer = tf.reshape(F3, [272, 1])

    # Output layer with linear activation
    #layer_1 = tf.reshape(layer_1, [sn, -1, n_hidden_vec1])
    #layer_1 = tf.transpose(layer_1, [1, 0, 2]) # batchsize, sn, hidden3
    #layer_1 = tf.reshape(layer_1, [-1, n_hidden_vec1]) # batchsize * sn, hidden3
    
    out_layer = tf.add(tf.matmul(layer_3, weights['out']), biases['bout'])  # 207*100, horizon
    out_layer = tf.reshape(out_layer, [sn, -1, horizon]) # 207, 100, horizon
    out_layer = tf.transpose(out_layer, [1, 0, 2]) # 100, 207, horizon
    out_layer = tf.reshape(out_layer, [-1, sn*horizon]) # 100, 207*horizon
    #tf.matmul(Atem1, layer_1)#tf.matmul(Atem, layer_3)
    #out_layer = tf.add(tf.matmul(Z4, weights['out']), biases['bout'])
    #out_layer = tf.nn.relu(out_layer)
    # weather layer 1
    #x_wea_tem = tf.reshape(x_wea[i,:], [1, 9*frequency2]) # 1 by 126
    #layer_1_wea = tf.add(tf.matmul(x_wea_tem, weights['h1_wea']), biases['b1_wea'])
    #layer_1_wea = tf.nn.relu(layer_1_wea)

    #out_layer_wea = tf.add(tf.matmul(layer_1_wea, weights['out_wea']), biases['bout_wea'])
    #out_layer = tf.add(out_layer, tf.reshape(out_layer_wea, [272, 1]))


    
    return out_layer

In [18]:
def gcn_corr_final(frequency, horizon, learning_rate, decay,batch_size, n_hidden_vec1,n_hidden_vec2,n_hidden_vec3,keep, early_stop_th,training_epochs, reg1, reg2):
    # set size
    #sn = 3 # station number
    X_whole = []
    Y_whole = []

    x_offsets = np.sort(
        # np.concatenate(([-week_size + 1, -day_size + 1], np.arange(-11, 1, 1)))
        np.concatenate((np.arange(-frequency+1, 1, 1),))
    )
    # Predict the next one hour
    y_offsets = np.sort(np.arange(1, 1+ horizon, 1))

    min_t = abs(min(x_offsets))
    max_t = abs(raw_data.shape[0] - abs(max(y_offsets)))  # Exclusive
    for t in range(min_t, max_t):
        x_t = raw_data.iloc[t + x_offsets, 0:sn].values.flatten('F')
        y_t = raw_data.iloc[t + y_offsets, 0:sn].values.flatten('F')
        X_whole.append(x_t)
        Y_whole.append(y_t)

    X_whole = np.stack(X_whole, axis=0)
    Y_whole = np.stack(Y_whole, axis=0)
    
    
    '''
    time_step = int(time_step) #

    #i = time_step
    #X_whole = np.zeros(shape = (raw_data.shape[0] - time_step, sn*time_step), dtype = np.float)
    #Y_whole = np.zeros(shape = (raw_data.shape[0] - time_step, sn), dtype = np.float)

    while i < raw_data.shape[0]:
        X_whole[i - time_step, ] = raw_data.iloc[(i - time_step):i, 0:sn].values.flatten('F') # 'F' flatten by column, default:flatten by row 0, 1, 2...7
        Y_whole[i - time_step, ] = raw_data.iloc[i, 0:sn]
        i = i + 1
    '''
    time_step = int(X_whole.shape[1] / sn)

    n_input = sn # station number
    n_input_vec = n_input * frequency # 207 * frequency
    n_A_vec = n_input * n_input
    n_output_vec = Y_whole.shape[1] # each row represent a result
    #print (n_output_vec)

    X_whole = np.reshape(X_whole, [X_whole.shape[0], sn, time_step])
    num_samples = X_whole.shape[0]
    num_test = round(num_samples * 0.2)
    num_train = round(num_samples * 0.7)
    num_val = num_samples - num_test - num_train
    #skip = skip1 + freq_max - time_step#time_step_max - time_step # to make sure the testing datasets are the same although the frequency could be different

    X_training = X_whole[:num_train, :]
    Y_training = Y_whole[:num_train, :]
    
    # shuffle
    perm = np.arange(X_training.shape[0])
    np.random.shuffle(perm)
    X_training = X_training[perm]
    Y_training = Y_training[perm]
    
    #print (type(X_training))
    #X_training = random.Random(6).shuffle(X_training)
    #Y_training = random.Random(6).shuffle(Y_training)

    X_val = X_whole[num_train:num_train+num_val, :]
    Y_val = Y_whole[num_train:num_train+num_val, :]
    #A_val = A_whole[0+training:0+training+validation, :]

    X_test = X_whole[-num_test:, :]
    Y_test = Y_whole[-num_test:, :]

    scaler = StandardScaler(mean=X_training.mean(), std=X_training.std())

    X_training = scaler.transform(X_training)
    Y_training = scaler.transform(Y_training)

    X_val = scaler.transform(X_val)
    Y_val = scaler.transform(Y_val)

    X_test = scaler.transform(X_test)
    Y_test = scaler.transform(Y_test)
    
    early_stop_th = int(early_stop_th)
    training_epochs = int(training_epochs)
    
    early_stop_k=0
    display_step = 1
    best_val = 10000
    traing_error = 0
    test_error = 0
    predic_res = []
    predic_step = []

    tf.reset_default_graph()

    keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)
    
    
    batch_size = int(batch_size)
    n_hidden_vec1 = int(n_hidden_vec1)
    n_hidden_vec2 = int(n_hidden_vec2)
    n_hidden_vec3 = int(n_hidden_vec3)
    early_stop_th = int(early_stop_th)
    training_epochs = int(training_epochs)
    
    early_stop_k=0
    display_step = 1
    best_val = 10000
    traing_error = 0
    test_error = 0
    # Network Parameters

    #n_classes = 2 # MNIST total classes (0-9 digits) # n_classes is for classification only

    tf.reset_default_graph()

    # tf Graph input
    X = tf.placeholder(tf.float32, [None, sn, time_step]) # X is the input signal
    #X_weather = tf.placeholder(tf.float32, [None, 9 * frequency2]) # X_weather weather and holiday information (9 is the feature number)
    A = tf.placeholder(tf.float32, [None, n_A_vec]) # A is the normalized adj matrix
    oldA = tf.placeholder(tf.float32, [n_input, n_input])
    Y = tf.placeholder(tf.float32, [None, n_output_vec]) # y is the regression output

    #Xtem = tf.placeholder(tf.float32, [n_input, frequency]) # for each row of X, A, Y, it can be reshaped to Xtem, Atem, Ytem
    #Atem = tf.placeholder(tf.float32, [n_input, n_input]) # 
    #Ytem = tf.placeholder(tf.float32, [n_input, 1]) #

    #Ypre = tf.placeholder(tf.float32, [None, n_output_vec])

    keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

    # Store layers weight & bias
    weights = {
        'h1': tf.Variable(tf.random_normal([frequency, n_hidden_vec1])),
        'h2': tf.Variable(tf.random_normal([n_hidden_vec1, n_hidden_vec2])),
        'h3': tf.Variable(tf.random_normal([n_hidden_vec2, n_hidden_vec3])),
        #'h4': tf.Variable(tf.random_normal([n_hidden_vec3, n_hidden_vec4])),
        'out': tf.Variable(tf.random_normal([n_hidden_vec3, horizon])), # dont forget to change n_hidden_vec1 when add/delete layers
        #'f1': tf.Variable(tf.random_normal([272*n_hidden_vec3, 100])),
        #'f2': tf.Variable(tf.random_normal([50, 10])),
        #'f3': tf.Variable(tf.random_normal([100, 272])),
        'A1': tf.Variable(tf.random_normal([n_input,n_input])),
        'A2': tf.Variable(tf.random_normal([n_input,n_input])),
        'A3': tf.Variable(tf.random_normal([n_input,n_input])),
        #'A4': tf.Variable(tf.random_normal([n_input,n_input])),
        #'h1_wea': tf.Variable(tf.random_normal([9*frequency2, n_hidden_weather1])),
        #'out_wea': tf.Variable(tf.random_normal([n_hidden_weather1, n_input]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([1,n_hidden_vec1])),# n_hidden_vec1])),# bias all the same??? dont forget to test
        'b2': tf.Variable(tf.random_normal([1, n_hidden_vec2])), #n_hidden_vec2])),
        'b3': tf.Variable(tf.random_normal([1, n_hidden_vec3])),#n_hidden_vec3])),
        #'b4': tf.Variable(tf.random_normal([n_input, n_hidden_vec4])),
        #'b1': tf.Variable(tf.random_normal([n_input,n_hidden_vec1])),# bias all the same??? dont forget to test
        #'b2': tf.Variable(tf.random_normal([n_input,n_hidden_vec2])),
        #'b3': tf.Variable(tf.random_normal([n_input,n_hidden_vec3])),
        #'bf1': tf.Variable(tf.random_normal([1, 100])), 
        #'bf2': tf.Variable(tf.random_normal([1, 10])), 
        #'bf3': tf.Variable(tf.random_normal([1, 272])), 
        'bout': tf.Variable(tf.random_normal([1, horizon])), 
        #'b1_wea': tf.Variable(tf.random_normal([1, n_hidden_weather1])), 
        #'bout_wea': tf.Variable(tf.random_normal([1, n_input])), 
    }

    #gcn_no_A
    #gcn
    # Construct model
    pred= gcn(X, weights, biases, batch_size,n_input, frequency, 1, n_output_vec)
    #print ('here!!!!!!!!!!!!!!!!')
    #set  negative in pred to 0s
    #Computes rectified linear: max(features, 0).
    #pred = tf.nn.relu(pred)
    # Define loss and optimizer
    # RMS for regression 
    # L2 regularization
    #cost = tf.reduce_mean(tf.pow(pred-Y, 2)) + reg1*tf.nn.l2_loss(weights['A1']) + reg2*tf.nn.l2_loss(weights['A2'])# + tf.nn.l2_loss(weights['A2']) + tf.nn.l2_loss(weights['A3'])))

    # L1 regular tf.reduce_sum(tf.abs(parameters))
    # mae
    
    pred = scaler.inverse_transform(pred)
    Y_true_tr = scaler.inverse_transform(Y)

    cost = masked_mae_tf(pred, Y_true_tr, 0)#+ reg1*tf.reduce_sum(tf.abs(weights['A1'])) +  reg2*tf.reduce_sum(tf.abs(weights['A2'])) + reg3*tf.reduce_sum(tf.abs(weights['A3']))#tf.reduce_mean(tf.abs(pred-Y_true_tr))# + reg1*tf.reduce_sum(tf.abs(weights['A1']))
    #cost = tf.reduce_mean(tf.pow(pred-Y, 2)) + reg1*tf.reduce_sum(tf.abs(weights['A1']))# + reg2*tf.reduce_sum(tf.abs(weights['A2']))
    pred_val= gcn(X, weights, biases, batch_size,n_input,frequency, 2, n_output_vec)
    #pred_val = tf.nn.relu(pred_val)
    #mae 
    pred_val = scaler.inverse_transform(pred_val)
    Y_true_val = scaler.inverse_transform(Y)
    cost_val = masked_mae_tf(pred_val, Y_true_val, 0)#tf.reduce_mean(tf.abs(pred_val-Y_true_val))
    #cost_val = tf.reduce_mean(tf.pow(pred_val-Y, 2))

    pred_tes= gcn(X, weights, biases, batch_size,n_input,frequency, 3, n_output_vec)
    #pred_tes = tf.nn.relu(pred_tes)
    # mae
    pred_tes = scaler.inverse_transform(pred_tes)
    Y_true_tes = scaler.inverse_transform(Y)
    '''
    predic_res_re = tf.reshape(pred_tes, [Y_test.shape[0], sn, horizon])
    test_Y_re = tf.reshape(Y_true_tes, [Y_test.shape[0], sn, horizon])
    
    cost_by_hor = []
    for hor_i in range(horizon):
        cost_by_hor.append(masked_mae_tf(predic_res_re[:, :, 0:hor_i], test_Y_re[:, :, 0:hor_i], 0))
    '''
    cost_tes = masked_mae_tf(pred_tes, Y_true_tes, 0)#tf.reduce_mean(tf.abs(pred_tes-Y_true_tes))
    #rmse
    #cost_tes = tf.reduce_mean(tf.pow(pred_tes-Y, 2))
    # cross-entropy for classification
    # cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y_train))
    # ratio = tf.abs(tf.reduce_sum(pred)-tf.reduce_sum(Y))/tf.reduce_sum(Y)
    #zero = 0
    #ratio = tf.reduce_mean(tf.divide(tf.where(tf.not_equal(Y, zero), np.abs(pred-Y), tf.zeros(Y.get_shape(), tf.float32)), tf.where(tf.not_equal(Y, zero), Y, tf.ones(Y.get_shape(), tf.float32))))
    #optimizer = tf.train.RMSPropOptimizer(learning_rate, decay).minimize(cost)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    # Initializing the variables
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

    #total_val_cost = []
    #total_val_ratio = []

    # learning start from 

    #index = daily_bike[(daily_bike['year'] == 2016) & (daily_bike['monthofyear'] == 1) & (daily_bike['dayofmonth'] == 1)].index.tolist()[0]
    #A_hat = normalize_adj(corr_matrix_trips)
    #print(A_hat)


    with tf.Session() as sess:
        sess.run(init)

        for epoch in range(training_epochs):

            avg_cost = 0.
            total_batch = int(num_train/batch_size)

            for i in range(total_batch):
                #print (Y_training[i*batch_size:(i+1)*batch_size,].size())
                _, c, preds, trueval = sess.run([optimizer, cost, pred, Y_true_tr], feed_dict={X: X_training[i*batch_size:(i+1)*batch_size,], 
                                                      Y: Y_training[i*batch_size:(i+1)*batch_size,], 
                                                              keep_prob: keep})
                #print (preds)
                #print (trueval)
                #print ("Epoch:", '%04d' % (epoch+1), "batch: ", i, "batch cost=", \
                #    "{:.9f}".format(c))
                #print ('here!!!!!!!!!!!!!!!!')
                avg_cost += c * batch_size #/ total_batch 
                #Display logs per epoch step
                
            # rest part of training dataset
            #num = num_train - total_batch*batch_size 
            if total_batch * batch_size != num_train:
                _, c, preds, trueval = sess.run([optimizer, cost, pred, Y_true_tr], feed_dict={X: X_training[total_batch*batch_size:num_train,], 
                                          Y: Y_training[total_batch*batch_size:num_train,],
                                                  keep_prob: keep})
                avg_cost += c * (num_train - total_batch*batch_size)
            
            avg_cost = avg_cost / num_train
            
            if epoch % display_step == 0:
                print ("Epoch:", '%04d' % (epoch+1), "Training MAE=", \
                    "{:.9f}".format(avg_cost)) #np.sqrt(avg_cost)
            # validation
            c_val = sess.run([cost_val], feed_dict={X: X_val, Y: Y_val,  keep_prob:1})
            print("Validation MAE: ", c_val[0])

            c_tes, pred_tes1, A1, Y_true = sess.run([cost_tes, pred_tes, weights['A1'], Y_true_tes], feed_dict={X: X_test,Y: Y_test, keep_prob: 1})
            print("Test MAE: ", c_tes)
            #print("predic step: ", cost_by_hor)

            if c_val[0] < best_val:
                best_val = c_val[0]
                #saver.save(sess, './bikesharing_graph_2_th_point1')
                test_error = c_tes
                traing_error = avg_cost#np.sqrt(avg_cost)
                early_stop_k = 0 # reset to 0
                #print (pred_tes1)
                predic_res = pred_tes1
                #predic_step = cost_by_hor

            # early stopping
            if c_val[0] >= best_val:
                early_stop_k += 1

            # threshold
            if early_stop_k == early_stop_th:
              #  print ("early stopping...")
                break
            

        print("epoch is ", epoch)
        print("training error is ", traing_error)
        print("Optimization Finished! the lowest validation MAE is ", best_val)#(np.sqrt(best_val)))
        print("The test MAE is ", test_error)#(np.sqrt(test_error)))
    
    test_Y = Y_test
    test_error = np.sqrt(test_error)
    return -best_val, predic_res,Y_true,test_error, A1#, predic_step#, A2

In [19]:
# increas batch size from 100 to 1000: validation error 39.XX at about 100 epochs
# batch size to 50: validation error 6.XX at about 100 epochs
# add one more gcn layer: 8.XX
# increase learning rate to 0.01, batch size is 100: 10.XX
# decrease learning rate to 0.005, batch size is 100: 10.XX
# decrease learning rate to 0.005, batch size is 50, two hidden levels (same matrix):
    # 6.XX at 100th epoch
# decrease learning rate to 0.005, batch size is 50, one hidden level, hidden 40:
    # 6.XX at 100th epoch
# decrease learning rate to 0.005, batch size is 50, one hidden level, hidden 20:
    # drop much faster
    # 6.XX can be get, but kind of overfitting at 100th epoch?
# decrease learning rate to 0.005, batch size is 50, one hidden level, hidden 10:
    # drop much faster
    #  at 100th epoch
# learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #reduce prediction horizon from 12 to 1, normalize the data:
    # val: 3.16

# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #reduce prediction horizon from 12 to 1, normalize the data, normalize the symmetric adjacency matrix:
    # train: 3.24, val: 3.42
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #reduce prediction horizon from 12 to 1, normalize the data, normalize the symmetric adjacency matrix, 
    # train: 3.08, val: 3.26
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #reduce prediction horizon from 12 to 1, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper
    # train: 2.82, val: 2.75
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, one layer
    # train: 4.08, val: 3.99, test: 4.29 
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, two layers (same adjmatrix)
    # train: 4.10, val: 4.19, test: 4.54 
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, two layers (different adjmatrix)
    # train: 3.52, val: 3.62, test: 4.03
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, two layers (different adjmatrix), bias set as a vector of same length
    # train: 3.53, val: 3.58, test: 3.91 
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, three layers (three adjmatrix), bias set as a vector of same length
    # batch from 50 to 100
    # train: 3.36, val: 3.42, test: 3.82
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, three layers (three adjmatrix), bias set as a vector of same length
    # batch from 50 to 100
    # last hidden size 10 to 20
    # train: 3.33, val: 3.38, test: 3.71
    # train: 3.24, val: 3.31, test: 3.66
    # train: 3.22, val: 3.29, test: 3.66
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, three layers (three adjmatrix), bias set as a vector of same length
    # batch from 50 to 100
    # three hidden size 5, 5, 10
    # train: 3.5, val: 3.48, test: 3.82 
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, three layers (three adjmatrix), bias set as a vector of same length
    # batch from 50 to 100
    # three hidden size 10, 10, 20
    # learning rate: 0.005
    # train: 3.23, val: 3.34, test: 3.74
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, three layers (three adjmatrix), bias set as a vector of same length
    # batch from 50 to 200
    # three hidden size 10, 10, 20
    # learning rate: 0.01
    # shuffle
    # train: 3.19, val: 3.30, test: 3.67
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, three layers (three adjmatrix), bias set as a vector of same length
    # batch from 50 to 100
    # three hidden size 10, 10, 20
    # learning rate: 0.01
    # shuffle
    # train: 3.15, val: 3.26, test: 3.61
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, three layers (three adjmatrix), bias set as a vector of same length
    # batch from 50 to 100
    # three hidden size 10, 10, 20
    # learning rate: 0.01
    # shuffle
    # fully freedom adjacency matrix (not symmetric )
    # train: , val: , test: 
    # training cost around 6.15, not decreasing anymore
    
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, three layers (three adjmatrix), bias set as a vector of same length
    # batch from 50 to 100
    # three hidden size 10, 10, 20
    # learning rate: 0.005
    # change gradient decent algorithm
    # train: 3.19, val: 3.28, test: 3.67

# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, three layers (three adjmatrix), bias set as a vector of same length
    # batch from 50 to 100
    # three hidden size 10, 10, 20
    # learning rate: 0.005
    # change gradient decent algorithm
    # keep: 0.8
    # train: 3.30, val: 3.36, test: 3.78
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, three layers (three adjmatrix), bias set as a vector of same length
    # batch from 50 to 200
    # three hidden size 10, 10, 20
    # learning rate: 0.01
    # change gradient decent algorithm
    # keep: 1
    # train: 3.15, val: 3.27, test: 3.63
    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, three layers (three adjmatrix), bias set as a vector of same length
    # batch from 50 to 200
    # three hidden size 10, 10, 20
    # learning rate: 0.01
    # change gradient decent algorithm
    # keep: 1
    # bias not the same size
    # decreasing slow

    
# increase sample, learning rate to 0.01, batch size is 50, one hidden level, hidden 10, 
    #prediction horizon 12, normalize the data, normalize the symmetric adjacency matrix, 
    # remove "0" ground truths as the paper, three layers (three adjmatrix), bias set as a vector of same length
    # batch size 200
    # second size 10 to 10
    # last hidden size 10 to 30
    # set the same size may lead to indecreasing cost (need to increase batch size)
    # train: 3.05, val: 3.22, test: 3.73

# batch size 150 really slow


In [20]:
# change hidden number
# change batch_size
# set size
#from bayes_opt import BayesianOptimization
import datetime
#freq_max = 12
#time_step = 12
learning_rate = 0.01
decay = 0.9 
batch_size = 500

early_stop_th = 150
training_epochs = 1000
keep = 1#0.2
#time_step_max = 10

sn = 207 # station num
#test = 2000 
reg1 = 0.05#0.05
reg2 = 0.05#0.1
reg3 = 0.05
frequency = 12
horizon = 12
n_hidden_vec1 = 10
n_hidden_vec2 = 10
n_hidden_vec3 = 20
n_hidden_vec4 = 10
#All_pred = np.empty([2000, 207])
#All_Y = np.empty([2000, 207])

#24*90
#step = 0
#gap = 100
#training = 0.7
#validation = 0.1
#test = 0.2

#gcn_corr_eval(7, 0.01, 0.5, 100, 0.4, 10, 5, 5, 0.2, 50, 500)



rep = 1 # repeating times

#total_sn = 0
#num_iter = 50
#init_points = 200


# stdbscan
#spatial_threshold = 300
#temporal_threshold = 300
#min_neighbors = 1 # number of neighbor

#frequency2 = skip1 + freq_max + training

#while step < 2000:

#hourly_bike_cluster = hourly_bike
best = -10000
pre_best = []
test_Y_best = []
test_error_best = 1000
A1_best = []
# A2_best = []
for i in range(rep):
    a = datetime.datetime.now()
    val_error, predic_res, test_Y,test_error, A1=gcn_corr_final(frequency, horizon, learning_rate, decay, batch_size, n_hidden_vec1,
                                                                n_hidden_vec2, n_hidden_vec3, keep, early_stop_th, training_epochs, reg1, reg2)
    #val_error, predic_res, test_Y,test_error=gcn_corr_final(a['frequency'], a['learning_rate'], a['decay'], a['batch_size'], a['n_hidden_vec1'], a['n_hidden_vec2'], a['n_hidden_vec3'], a['keep'], a['early_stop_th'], a['training_epochs'], a['reg'])
    #print ("finished A running: ", i)
    b = datetime.datetime.now()
    print(b-a)
    if val_error > best:
        best = val_error
        pre_best = predic_res
        test_Y_best = test_Y
        test_error_best = test_error
        #A1_best = A1
       # A2_best = A2

    #val_error, predic_res, test_Y,test_error=gcn_corr_final(24, 0.02, 0.2, 100, 5, 10, 10, 0.8, 50, 500)
    All_pred = pre_best
    All_Y  = test_Y_best

    #total_sn = total_sn + sn

    #total_error = np.sqrt(np.mean((All_pred[0:(step+gap),0:total_sn] - All_Y[0:(step+gap),0:total_sn])**2))

    #print ("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!the cluster now is: ", c)
    #print ("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!the val error of this cluster now is: ", best)
    #print ("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!the test error by this cluster now is: ", total_error)
    #step = step + gap
    #skip1 = skip1 + gap
    
    #np.savetxt("prediction_2.csv", All_pred, delimiter = ',')
    #np.savetxt("prediction_Y_2.csv", All_Y, delimiter = ',')
    

Epoch: 0001 Training MAE= 123.990211574
Validation MAE:  20.14675
Test MAE:  21.65484
Epoch: 0002 Training MAE= 13.750733587
Validation MAE:  11.100066
Test MAE:  12.026818
Epoch: 0003 Training MAE= 9.664486127
Validation MAE:  8.867375
Test MAE:  9.639506
Epoch: 0004 Training MAE= 8.052066818
Validation MAE:  7.649607
Test MAE:  8.391342
Epoch: 0005 Training MAE= 7.177701311
Validation MAE:  7.083384
Test MAE:  7.79229
Epoch: 0006 Training MAE= 6.813359119
Validation MAE:  6.824097
Test MAE:  7.522514
Epoch: 0007 Training MAE= 6.631802164
Validation MAE:  6.667398
Test MAE:  7.3559117
Epoch: 0008 Training MAE= 6.505465001
Validation MAE:  6.533195
Test MAE:  7.213554
Epoch: 0009 Training MAE= 6.395922012
Validation MAE:  6.4169025
Test MAE:  7.0872974
Epoch: 0010 Training MAE= 6.301142514
Validation MAE:  6.3131833
Test MAE:  6.9805727
Epoch: 0011 Training MAE= 6.210150160
Validation MAE:  6.193912
Test MAE:  6.862125
Epoch: 0012 Training MAE= 6.105813477
Validation MAE:  6.051965
Tes

Epoch: 0098 Training MAE= 3.798085485
Validation MAE:  3.8476582
Test MAE:  4.337153
Epoch: 0099 Training MAE= 3.788126507
Validation MAE:  3.8061311
Test MAE:  4.295178
Epoch: 0100 Training MAE= 3.792393641
Validation MAE:  3.804786
Test MAE:  4.281302
Epoch: 0101 Training MAE= 3.784581155
Validation MAE:  3.7957048
Test MAE:  4.2782354
Epoch: 0102 Training MAE= 3.790763546
Validation MAE:  3.831224
Test MAE:  4.3190036
Epoch: 0103 Training MAE= 3.777169967
Validation MAE:  3.8291726
Test MAE:  4.3206277
Epoch: 0104 Training MAE= 3.772030086
Validation MAE:  3.7946415
Test MAE:  4.2828126
Epoch: 0105 Training MAE= 3.779356860
Validation MAE:  3.8006816
Test MAE:  4.295173
Epoch: 0106 Training MAE= 3.767780369
Validation MAE:  3.7988946
Test MAE:  4.2943954
Epoch: 0107 Training MAE= 3.762665841
Validation MAE:  3.814115
Test MAE:  4.2925777
Epoch: 0108 Training MAE= 3.766523892
Validation MAE:  3.77986
Test MAE:  4.26909
Epoch: 0109 Training MAE= 3.772756454
Validation MAE:  3.7935395


Epoch: 0195 Training MAE= 3.598214961
Validation MAE:  3.6017752
Test MAE:  4.139998
Epoch: 0196 Training MAE= 3.598769876
Validation MAE:  3.607224
Test MAE:  4.1434846
Epoch: 0197 Training MAE= 3.599005505
Validation MAE:  3.6210854
Test MAE:  4.1631556
Epoch: 0198 Training MAE= 3.599016434
Validation MAE:  3.6246955
Test MAE:  4.165989
Epoch: 0199 Training MAE= 3.590879836
Validation MAE:  3.5946543
Test MAE:  4.1341076
Epoch: 0200 Training MAE= 3.583844733
Validation MAE:  3.5954597
Test MAE:  4.131694
Epoch: 0201 Training MAE= 3.583103460
Validation MAE:  3.5910585
Test MAE:  4.1324897
Epoch: 0202 Training MAE= 3.589432724
Validation MAE:  3.6073768
Test MAE:  4.1439457
Epoch: 0203 Training MAE= 3.583529048
Validation MAE:  3.5909793
Test MAE:  4.1332765
Epoch: 0204 Training MAE= 3.594673559
Validation MAE:  3.5963864
Test MAE:  4.1363883
Epoch: 0205 Training MAE= 3.591215365
Validation MAE:  3.5906546
Test MAE:  4.126217
Epoch: 0206 Training MAE= 3.582353206
Validation MAE:  3.58

Epoch: 0292 Training MAE= 3.511410186
Validation MAE:  3.5165374
Test MAE:  4.0806584
Epoch: 0293 Training MAE= 3.508167719
Validation MAE:  3.5176892
Test MAE:  4.080048
Epoch: 0294 Training MAE= 3.508989432
Validation MAE:  3.5147576
Test MAE:  4.0779743
Epoch: 0295 Training MAE= 3.504474313
Validation MAE:  3.5216389
Test MAE:  4.0848613
Epoch: 0296 Training MAE= 3.500495527
Validation MAE:  3.5257106
Test MAE:  4.0862203
Epoch: 0297 Training MAE= 3.495225329
Validation MAE:  3.5388324
Test MAE:  4.089668
Epoch: 0298 Training MAE= 3.495489978
Validation MAE:  3.552229
Test MAE:  4.092698
Epoch: 0299 Training MAE= 3.496891622
Validation MAE:  3.5274303
Test MAE:  4.0792713
Epoch: 0300 Training MAE= 3.517047144
Validation MAE:  3.523317
Test MAE:  4.0862036
Epoch: 0301 Training MAE= 3.503529386
Validation MAE:  3.535359
Test MAE:  4.099552
Epoch: 0302 Training MAE= 3.493707390
Validation MAE:  3.558206
Test MAE:  4.108438
Epoch: 0303 Training MAE= 3.510713955
Validation MAE:  3.518339

Epoch: 0389 Training MAE= 3.453858497
Validation MAE:  3.491676
Test MAE:  4.058645
Epoch: 0390 Training MAE= 3.445546745
Validation MAE:  3.5001726
Test MAE:  4.072263
Epoch: 0391 Training MAE= 3.448139288
Validation MAE:  3.5079017
Test MAE:  4.075655
Epoch: 0392 Training MAE= 3.460588163
Validation MAE:  3.5016673
Test MAE:  4.072405
Epoch: 0393 Training MAE= 3.468047393
Validation MAE:  3.540707
Test MAE:  4.1093464
Epoch: 0394 Training MAE= 3.463109476
Validation MAE:  3.5241122
Test MAE:  4.0971084
Epoch: 0395 Training MAE= 3.452369964
Validation MAE:  3.4883895
Test MAE:  4.0537453
Epoch: 0396 Training MAE= 3.444385295
Validation MAE:  3.4971015
Test MAE:  4.0646834
Epoch: 0397 Training MAE= 3.440759452
Validation MAE:  3.491876
Test MAE:  4.0590544
Epoch: 0398 Training MAE= 3.442060276
Validation MAE:  3.4885871
Test MAE:  4.055683
Epoch: 0399 Training MAE= 3.481824274
Validation MAE:  3.5637968
Test MAE:  4.1444716
Epoch: 0400 Training MAE= 3.475072556
Validation MAE:  3.55526

...

Epoch: 0583 Training MAE= 3.378702009
Validation MAE:  3.448472
Test MAE:  4.0442734
Epoch: 0584 Training MAE= 3.378942022
Validation MAE:  3.4531324
Test MAE:  4.0481424
Epoch: 0585 Training MAE= 3.383312707
Validation MAE:  3.45059
Test MAE:  4.04531
Epoch: 0586 Training MAE= 3.399383428
Validation MAE:  3.4606414
Test MAE:  4.0454273
Epoch: 0587 Training MAE= 3.396964213
Validation MAE:  3.4788404
Test MAE:  4.064341
Epoch: 0588 Training MAE= 3.387770117
Validation MAE:  3.452789
Test MAE:  4.0435486
Epoch: 0589 Training MAE= 3.379893576
Validation MAE:  3.4528625
Test MAE:  4.045002
Epoch: 0590 Training MAE= 3.392705185
Validation MAE:  3.4604762
Test MAE:  4.0521607
Epoch: 0591 Training MAE= 3.391719852
Validation MAE:  3.4682753
Test MAE:  4.0551553
Epoch: 0592 Training MAE= 3.393966001
Validation MAE:  3.4670389
Test MAE:  4.049082
Epoch: 0593 Training MAE= 3.385638253
Validation MAE:  3.454231
Test MAE:  4.043989
Epoch: 0594 Training MAE= 3.383753101
Validation MAE:  3.4594946


Epoch: 0680 Training MAE= 3.378124442
Validation MAE:  3.4400384
Test MAE:  4.0530868
Epoch: 0681 Training MAE= 3.375187325
Validation MAE:  3.4354537
Test MAE:  4.050964
Epoch: 0682 Training MAE= 3.370306543
Validation MAE:  3.4288678
Test MAE:  4.0413275
Epoch: 0683 Training MAE= 3.367162133
Validation MAE:  3.4261377
Test MAE:  4.042942
Epoch: 0684 Training MAE= 3.365508987
Validation MAE:  3.4255018
Test MAE:  4.040661
Epoch: 0685 Training MAE= 3.363607303
Validation MAE:  3.4313474
Test MAE:  4.0518737
Epoch: 0686 Training MAE= 3.368621872
Validation MAE:  3.4276583
Test MAE:  4.0367327
Epoch: 0687 Training MAE= 3.367563640
Validation MAE:  3.4276752
Test MAE:  4.037343
Epoch: 0688 Training MAE= 3.360158742
Validation MAE:  3.4254456
Test MAE:  4.0389957
Epoch: 0689 Training MAE= 3.353885134
Validation MAE:  3.426425
Test MAE:  4.0358086
Epoch: 0690 Training MAE= 3.353435421
Validation MAE:  3.4284084
Test MAE:  4.03646
Epoch: 0691 Training MAE= 3.351866168
Validation MAE:  3.4291

Epoch: 0776 Training MAE= 3.342916944
Validation MAE:  3.4284883
Test MAE:  4.0328894
Epoch: 0777 Training MAE= 3.351417240
Validation MAE:  3.425878
Test MAE:  4.0291095
Epoch: 0778 Training MAE= 3.360204266
Validation MAE:  3.4350593
Test MAE:  4.037645
Epoch: 0779 Training MAE= 3.354015666
Validation MAE:  3.4249165
Test MAE:  4.0228653
Epoch: 0780 Training MAE= 3.345104480
Validation MAE:  3.4243615
Test MAE:  4.0206337
Epoch: 0781 Training MAE= 3.340789752
Validation MAE:  3.4214897
Test MAE:  4.021012
Epoch: 0782 Training MAE= 3.341903952
Validation MAE:  3.4212468
Test MAE:  4.025272
Epoch: 0783 Training MAE= 3.351440430
Validation MAE:  3.4377096
Test MAE:  4.0316033
Epoch: 0784 Training MAE= 3.352268477
Validation MAE:  3.4318612
Test MAE:  4.031846
Epoch: 0785 Training MAE= 3.348260419
Validation MAE:  3.421703
Test MAE:  4.021438
Epoch: 0786 Training MAE= 3.339682070
Validation MAE:  3.4221509
Test MAE:  4.019037
Epoch: 0787 Training MAE= 3.351776192
Validation MAE:  3.43253

Test MAE:  4.02964
Epoch: 0873 Training MAE= 3.320278462
Validation MAE:  3.4227035
Test MAE:  4.0191083
Epoch: 0874 Training MAE= 3.325115981
Validation MAE:  3.4332185
Test MAE:  4.0392957
Epoch: 0875 Training MAE= 3.325323164
Validation MAE:  3.423615
Test MAE:  4.033794
Epoch: 0876 Training MAE= 3.326362755
Validation MAE:  3.463124
Test MAE:  4.086125
Epoch: 0877 Training MAE= 3.349543803
Validation MAE:  3.4850647
Test MAE:  4.0573034
Epoch: 0878 Training MAE= 3.380113198
Validation MAE:  3.476124
Test MAE:  4.077391
Epoch: 0879 Training MAE= 3.355803090
Validation MAE:  3.4223359
Test MAE:  4.0244484
Epoch: 0880 Training MAE= 3.342521046
Validation MAE:  3.419907
Test MAE:  4.017391
Epoch: 0881 Training MAE= 3.324024882
Validation MAE:  3.4304838
Test MAE:  4.022241
Epoch: 0882 Training MAE= 3.320744923
Validation MAE:  3.4206774
Test MAE:  4.0114727
Epoch: 0883 Training MAE= 3.315609789
Validation MAE:  3.4212377
Test MAE:  4.0136423
Epoch: 0884 Training MAE= 3.315894366
Valida

KeyboardInterrupt: 

In [ ]:
All_pred_tf = tf.convert_to_tensor(
    All_pred,
    dtype=np.float32,
    name=None,
    preferred_dtype=None
)

#All_Y = scaler.inverse_transform(All_Y)

All_Y_tf =  tf.convert_to_tensor(
    All_Y,
    dtype=np.float32,
    name=None,
    preferred_dtype=None
)

predic_res_re = tf.reshape(All_pred_tf, [All_pred.shape[0], sn, horizon])

test_Y_re = tf.reshape(All_Y_tf, [All_pred.shape[0], sn, horizon])

cost_by_hor = []
with tf.Session() as sess:  
    for hor_i in range(horizon):
        cost_by_hor.append(masked_mae_tf(predic_res_re[:, :, 0:hor_i+1], test_Y_re[:, :, 0:hor_i+1], 0).eval())

In [ ]:
cost_by_hor

In [69]:
np.savetxt("../../data/trajectory/gcnn_prediction.csv", All_pred, delimiter = ',')
np.savetxt("../../data/trajectory/gcnn_prediction_Y.csv", All_Y, delimiter = ',')

In [70]:
All_pred_tf.shape

TensorShape([Dimension(6850), Dimension(2484)])